In [1]:
import os, glob
import torch
import pandas as pd
import numpy as np

/data/lesc/users/rustichini/miniconda3/envs/jpeg_ai_vm/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV

In [9]:
from train_RF import get_name, create_dataset

In [4]:
dir_train = "/data/lesc/users/rustichini/thesis/output/medium_dataset_output/target_bpp_6/train/"
df_train = pd.read_csv("/data/lesc/users/rustichini/thesis/train.csv")
df_train = df_train.drop(columns = ["original_path", "path"])



dir_test = "/data/lesc/users/rustichini/thesis/output/medium_dataset_output/target_bpp_6/test/"
df_test = pd.read_csv("/data/lesc/users/rustichini/thesis/test.csv")
df_test = df_test.drop(columns = ["original_path", "path"])

X_train, y_train = create_dataset(df_train, dir_train)
X_test, y_test = create_dataset(df_test, dir_test)

In [13]:
print("(num_samples, num_features)")
print(f"Train dataset : {X_train.shape}")
print(f"Test dataset: {X_test.shape}")

(num_samples, num_features)
Train dataset : (2000, 65536)
Test dataset: (200, 65536)


In [5]:
rf = RandomForestClassifier(n_estimators=250)
rf.fit(X_train,y_train)

RandomForestClassifier(n_estimators=250)

In [6]:
y_pred = rf.predict(X_test)
y_proba = rf.predict_proba(X_test)

In [7]:
# --- Valutazione delle Prestazioni ---
print("\n--- Valutazione delle Prestazioni del Random Forest ---")

num_classes = len(np.unique(y_train))
class_names = [f'Classe {i}' for i in range(num_classes)]

accuracy = accuracy_score(y_test, y_pred)
print(f"1. Accuratezza sul Test Set: {accuracy:.4f}")


--- Valutazione delle Prestazioni del Random Forest ---
1. Accuratezza sul Test Set: 0.7700
